In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, chi2_contingency, ks_2samp
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [22]:
df = pd.read_excel('/content/Telco-Customer-Churn.xlsx')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [23]:
from exploracion import calidad_datos

In [24]:
resumen = calidad_datos(df)


In [25]:
resumen

,tipo,nan,porcentaje_nan,ceros,porcentaje_ceros,count,unique,top,freq,mean,std,min,25%,50%,75%,max,IQR,lim_inf,lim_sup,atipicos
SeniorCitizen,int64,0,0.000000,5901,83.785319,7043.0,NaN,NaN,NaN,0.162147,0.368612,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1142
tenure,int64,0,0.000000,11,0.156183,7043.0,NaN,NaN,NaN,32.371149,24.559481,0.0,9.0,29.0,55.0,72.0,46.0,-60.0,124.0,0
MonthlyCharges,float64,0,0.000000,0,0.000000,7043.0,NaN,NaN,NaN,64.761692,30.090047,18.25,35.5,70.35,89.85,118.75,54.35,-46.025,171.375,0
TotalCharges,float64,11,0.156183,0,0.000000,7032.0,NaN,NaN,NaN,2283.300441,2266.771362,18.8,401.45,1397.475,3794.7375,8684.8,3393.2875,-4688.48125,8884.66875,0
customerID,object,0,0.000000,0,0.000000,7043,7043,3186-AJIEK,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
PaymentMethod,object,0,0.000000,0,0.000000,7043,4,Electronic check,2365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
PaperlessBilling,object,0,0.000000,0,0.000000,7043,2,Yes,4171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
Contract,object,0,0.000000,0,0.000000,7043,3,Month-to-month,3875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
StreamingMovies,object,0,0.000000,0,0.000000,7043,3,No,2785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
StreamingTV,object,0,0.000000,0,0.000000,7043,3,No,2810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


Cantidad total de registros: 7,043
Número de columnas: 20 (algunas numéricas y otras categóricas)
Valores nulos: Solo la columna TotalCharges tiene 11 valores nulos (0.16% del total).
Valores cero: SeniorCitizen tiene un 83.79% de valores en cero (posiblemente representa a los clientes que no son adultos mayores).
tenure (tiempo de permanencia) tiene un 0.15% en cero (clientes nuevos).
Otras columnas numéricas no presentan ceros significativos, en cuanto a las variables numericas SeniorCitizen es binaria (0 = no, 1 = sí).
TotalCharges tiene valores atípicos negativos según los límites IQR, pero no hay registros detectados como outliers.
tenure no presenta outliers.
Variables Categóricas
Alta cardinalidad: customerID es única para cada cliente.
Más categorías comunes:
PaymentMethod: 4 métodos, el más usado es Electronic Check (2,365 clientes).
Contract: 3 tipos, Month-to-Month es el más común (3,875 clientes).
Churn: 2 valores (Sí/No), con más clientes que no cancelaron (5,174 vs. 1,869).

# 2. Preprocesamiento de datos

In [26]:
# 1. Manejo de valores nulos
df = df.dropna()  # Elimina filas con valores nulos

# 2. Conversión de columnas numéricas incorrectas
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna()  # Elimina filas que quedaron con NaN después de la conversión

# 3. Eliminación de columnas irrelevantes (ID del cliente)
df = df.drop(columns=['customerID'])

# 4. Codificación de variables categóricas
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# 5. Normalización de características numéricas
scaler = StandardScaler()
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# 6. Verificación final del dataset
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7032 non-null   int64  
 1   SeniorCitizen     7032 non-null   int64  
 2   Partner           7032 non-null   int64  
 3   Dependents        7032 non-null   int64  
 4   tenure            7032 non-null   float64
 5   PhoneService      7032 non-null   int64  
 6   MultipleLines     7032 non-null   int64  
 7   InternetService   7032 non-null   int64  
 8   OnlineSecurity    7032 non-null   int64  
 9   OnlineBackup      7032 non-null   int64  
 10  DeviceProtection  7032 non-null   int64  
 11  TechSupport       7032 non-null   int64  
 12  StreamingTV       7032 non-null   int64  
 13  StreamingMovies   7032 non-null   int64  
 14  Contract          7032 non-null   int64  
 15  PaperlessBilling  7032 non-null   int64  
 16  PaymentMethod     7032 non-null   int64  
 17  

# 3. Dividir los datos en conjuntos de entrenamiento y prueba.

In [27]:
from sklearn.model_selection import train_test_split

# Definir características (X) y variable objetivo (y)
X = df.drop(columns=['Churn'])  # Eliminamos la columna objetivo
y = df['Churn']  # Variable objetivo

# Dividir en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Verificar el tamaño de los conjuntos
print(f"Tamaño de X_train: {X_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}")
print(f"Distribución de 'Churn' en train:\n{y_train.value_counts(normalize=True)}")
print(f"Distribución de 'Churn' en test:\n{y_test.value_counts(normalize=True)}")

Tamaño de X_train: (5625, 19)
Tamaño de X_test: (1407, 19)
Distribución de 'Churn' en train:
Churn
0    0.734222
1    0.265778
Name: proportion, dtype: float64
Distribución de 'Churn' en test:
Churn
0    0.734186
1    0.265814
Name: proportion, dtype: float64


Los conjuntos de entrenamiento y prueba están correctamente divididos, y la variable Churn mantiene su proporción en ambos (≈ 73.4% clientes que no se fugan y ≈ 26.6% que sí). Esto confirma que la estratificación funcionó bien.

# 4. Entrenar al menos 3 algoritmos y optimizar sus hiperparametros


In [28]:
# 1. Definimos los modelos
logreg = LogisticRegression(max_iter=1000)
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()

# 2. Definimos los parámetros a optimizar para cada modelo
param_grid_logreg = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga'],
    'penalty': ['l1', 'l2']
}

param_grid_dt = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 3. Aplicamos GridSearchCV para optimizar los hiperparámetros
grid_logreg = GridSearchCV(estimator=logreg, param_grid=param_grid_logreg, cv=5, n_jobs=-1, verbose=1)
grid_dt = GridSearchCV(estimator=dt, param_grid=param_grid_dt, cv=5, n_jobs=-1, verbose=1)
grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=1)

# 4. Entrenamos los modelos con los datos de entrenamiento
grid_logreg.fit(X_train, y_train)
grid_dt.fit(X_train, y_train)
grid_rf.fit(X_train, y_train)

# 5. Evaluamos el rendimiento de los modelos
y_pred_logreg = grid_logreg.predict(X_test)
y_pred_dt = grid_dt.predict(X_test)
y_pred_rf = grid_rf.predict(X_test)

# 6. Mostramos los resultados
print("Mejores parámetros para Regresión Logística:", grid_logreg.best_params_)
print("Mejores parámetros para Árbol de Decisión:", grid_dt.best_params_)
print("Mejores parámetros para Random Forest:", grid_rf.best_params_)

print("\nReporte de clasificación para Regresión Logística:")
print(classification_report(y_test, y_pred_logreg))

print("\nReporte de clasificación para Árbol de Decisión:")
print(classification_report(y_test, y_pred_dt))

print("\nReporte de clasificación para Random Forest:")
print(classification_report(y_test, y_pred_rf))

# 7. Exactitud de los modelos
print("\nExactitud para Regresión Logística:", accuracy_score(y_test, y_pred_logreg))
print("Exactitud para Árbol de Decisión:", accuracy_score(y_test, y_pred_dt))
print("Exactitud para Random Forest:", accuracy_score(y_test, y_pred_rf))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Mejores parámetros para Regresión Logística: {'C': 0.01, 'penalty': 'l2', 'solver': 'saga'}
Mejores parámetros para Árbol de Decisión: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10}
Mejores parámetros para Random Forest: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}

Reporte de clasificación para Regresión Logística:
              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1033
           1       0.62      0.50      0.55       374

    accuracy                           0.79      1407
   macro avg       0.73      0.69      0.71      1407
weighted avg       0.78      0.79      0.78      1407


Reporte de clasificación para Árbol de Decisión:
              precision    recall  f1-

Regresión Logística:
Exactitud: 78.68%
El modelo tiene una precisión de 0.83 para los clientes que no se fugan (0), pero solo una recall de 0.50 para los clientes fugados (1).
Esto indica que, aunque es bueno para predecir clientes que no se fugan, le cuesta identificar a aquellos que sí se fugan.
Árbol de Decisión:
Exactitud: 78.18%
Similar a la regresión logística en términos de precisión y recall, con un desempeño algo inferior al de la regresión logística en la exactitud general.
Random Forest:
Exactitud: 79.46%
Este modelo tiene un buen balance, con una precisión de 0.83 para los clientes no fugados y una recall de 0.50 para los clientes fugados. Aunque el modelo tiene un buen desempeño general, aún tiene dificultades para identificar correctamente a los clientes que se fugarán.

# 5. Evaluacion de metricas

Regresión Logística:
Precisión: 0.83 (para la clase 0 - clientes no fugados)
Sensibilidad: 0.50 (para la clase 1 - clientes fugados)
F1-score: 0.55 (para la clase 1 - clientes fugados)

Árbol de Decisión:
Precisión: 0.83 (para la clase 0)
Sensibilidad: 0.50 (para la clase 1)
F1-score: 0.55 (para la clase 1)

Random Forest:
Precisión: 0.83 (para la clase 0)
Sensibilidad: 0.50 (para la clase 1)
F1-score: 0.56 (para la clase 1)

Área bajo la curva ROC (AUC-ROC): Una métrica que evalúa la capacidad del modelo para distinguir entre las clases positivas y negativas.

In [30]:
# Entrenar (ajustar) el modelo con los mejores parámetros encontrados
logreg = LogisticRegression(C=0.01, penalty='l2', solver='saga')
logreg.fit(X_train, y_train)

# Predicciones de probabilidades para AUC
y_pred_prob_logistic = logreg.predict_proba(X_test)[:, 1]

In [31]:
# Entrenar Árbol de Decisión con los mejores parámetros
tree = DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_leaf=2, min_samples_split=10)
tree.fit(X_train, y_train)

# Predicciones de probabilidades para AUC
y_pred_prob_tree = tree.predict_proba(X_test)[:, 1]

# Entrenar Random Forest con los mejores parámetros
rf = RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=2, n_estimators=150)
rf.fit(X_train, y_train)

# Predicciones de probabilidades para AUC
y_pred_prob_rf = rf.predict_proba(X_test)[:, 1]

In [32]:
# Predicciones del modelo (probabilidades para AUC)
y_pred_prob_logistic = logreg.predict_proba(X_test)[:, 1]
y_pred_prob_tree = tree.predict_proba(X_test)[:, 1]
y_pred_prob_rf = rf.predict_proba(X_test)[:, 1]

# Cálculo de AUC-ROC
auc_logistic = roc_auc_score(y_test, y_pred_prob_logistic)
auc_tree = roc_auc_score(y_test, y_pred_prob_tree)
auc_rf = roc_auc_score(y_test, y_pred_prob_rf)

# Cálculo de la matriz de confusión para obtener la Especificidad
cm_logistic = confusion_matrix(y_test, logreg.predict(X_test))
cm_tree = confusion_matrix(y_test, tree.predict(X_test))
cm_rf = confusion_matrix(y_test, rf.predict(X_test))

# Especificidad = TN / (TN + FP)
specificity_logistic = cm_logistic[0, 0] / (cm_logistic[0, 0] + cm_logistic[0, 1])
specificity_tree = cm_tree[0, 0] / (cm_tree[0, 0] + cm_tree[0, 1])
specificity_rf = cm_rf[0, 0] / (cm_rf[0, 0] + cm_rf[0, 1])

# Resultados
print(f"AUC-ROC para Regresión Logística: {auc_logistic}")
print(f"AUC-ROC para Árbol de Decisión: {auc_tree}")
print(f"AUC-ROC para Random Forest: {auc_rf}")

print(f"Especificidad para Regresión Logística: {specificity_logistic}")
print(f"Especificidad para Árbol de Decisión: {specificity_tree}")
print(f"Especificidad para Random Forest: {specificity_rf}")

AUC-ROC para Regresión Logística: 0.8288524157352812
AUC-ROC para Árbol de Decisión: 0.8188612679957138
AUC-ROC para Random Forest: 0.8333225484156525
Especificidad para Regresión Logística: 0.8915779283639884
Especificidad para Árbol de Decisión: 0.8838334946757018
Especificidad para Random Forest: 0.8993223620522749


La AUC-ROC mide el rendimiento de un modelo de clasificación binaria al representar su capacidad para discriminar entre las clases positivas y negativas en varios umbrales de probabilidad. Un valor cercano a 1 indica un buen desempeño y un valor cercano a 0.5 sugiere que el modelo no tiene capacidad discriminatoria. Aquí están tus resultados:

Regresión Logística: 0.83
Árbol de Decisión: 0.82
Random Forest: 0.83

En este caso, los tres modelos tienen un AUC-ROC bastante alto, lo que indica que todos tienen una buena capacidad para predecir las probabilidades de fuga de clientes.

La especificidad mide la proporción de verdaderos negativos que el modelo puede identificar correctamente. Es útil cuando las clases están desequilibradas, ya que muestra cuántos de los clientes que no se van (no fugados) son correctamente identificados. Los resultados de especificidad son:

Regresión Logística: 0.89
Árbol de Decisión: 0.88
Random Forest: 0.90
Estos resultados muestran que el Random Forest tiene la mayor especificidad, es decir, es el mejor para identificar a los clientes que no se van (no fugados) correctamente, seguido de la Regresión Logística.

AUC-ROC: El Random Forest tiene ligeramente la mejor AUC-ROC (0.83), lo que sugiere que podría ser el modelo más fuerte en cuanto a predicción de la probabilidad de fuga de clientes.
Especificidad: El Random Forest también tiene la mejor especificidad, lo que indica que puede ser mejor para identificar clientes que no se van.
En términos generales, si la prioridad es identificar a los clientes que están en riesgo de fuga (clase 1), entonces el AUC-ROC es la métrica más relevante. Si el objetivo es minimizar el número de falsos positivos (clientes que no se van, pero son identificados como de riesgo), entonces la especificidad sería clave.